<a href="https://colab.research.google.com/github/mcadete/LLM_4_Biz_16/blob/main/MC_Tech16_HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai

In [ ]:
from openai import OpenAI
from google.colab import userdata

open_ai_key = userdata.get('open_ai_key')
client = OpenAI(api_key=open_ai_key)

In [ ]:
# Get a PDF on tax loss harvesting
!wget https://corporate.vanguard.com/content/dam/corp/research/pdf/tax_loss_harvesting_why_a_personalized_approach_is_important.pdf -O hw2.pdf

--2025-02-12 05:55:48--  https://corporate.vanguard.com/content/dam/corp/research/pdf/tax_loss_harvesting_why_a_personalized_approach_is_important.pdf
Resolving corporate.vanguard.com (corporate.vanguard.com)... 23.60.96.50, 23.60.96.47
Connecting to corporate.vanguard.com (corporate.vanguard.com)|23.60.96.50|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 461908 (451K) [application/pdf]
Saving to: ‘hw2.pdf’

hw2.pdf             100%[===================>] 451.08K   824KB/s    in 0.5s    

2025-02-12 05:55:49 (824 KB/s) - ‘hw2.pdf’ saved [461908/461908]



In [ ]:
!ls

hw2.pdf  sample_data


In [ ]:
!pip install -U langchain-community pypdf langchain-openai

In [ ]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("hw2.pdf")
pages = loader.load_and_split()

In [ ]:
len(pages)

16

In [ ]:
pages[0:16]

[Document(metadata={'producer': 'Adobe PDF Library 17.0', 'creator': 'Adobe InDesign 19.3 (Macintosh)', 'creationdate': '2024-08-28T05:51:08-04:00', 'author': 'Vanguard', 'moddate': '2024-08-28T07:10:40-04:00', 'subject': 'Vanguard research on setting expectations and maximizing the probability of success when using a tax-loss-harvesting strategy', 'title': 'Tax-loss harvesting: Why a personalized approach is important', 'trapped': '/False', 'source': 'hw2.pdf', 'total_pages': 16, 'page': 0, 'page_label': '1'}, page_content='Vanguard research July 2024\nTax-loss harvesting: Why a \npersonalized approach is important\n ● The value of tax-loss harvesting (TLH) varies significantly across investor \ncharacteristics, investor behavior, and market environment. Each of these  \nthree categories drives roughly one-third of the variation in value. \n ● We consider several net-worth-based profiles informed by the Survey of \nConsumer Finances. Every profile analyzed showed improvement in TLH \n

In [ ]:
from langchain.chains.summarize import load_summarize_chain
from langchain_community.document_loaders import WebBaseLoader
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0.1, model_name="gpt-4-turbo-preview", api_key=open_ai_key)
chain = load_summarize_chain(llm, chain_type="stuff")

res = chain.invoke(pages[0:16])
res
print(res["output_text"])

The Vanguard research from July 2024 emphasizes the importance of a personalized approach to tax-loss harvesting (TLH), highlighting that the value of TLH varies significantly due to investor characteristics, behaviors, and market conditions. The study, using net-worth-based profiles, demonstrates that optimal TLH behavior, especially the reinvestment of tax savings, significantly improves outcomes across all profiles, with potential TLH alpha ranging between 0.47% and 1.27%. The research underscores that TLH's effectiveness is influenced equally by factors within the investor's control, specific to the investor, and beyond the investor's control. Risks associated with TLH, such as wash sales and ossification, are acknowledged, alongside the strategy's dependency on market performance and tax rate changes. The study concludes that while TLH can offer substantial benefits, especially for high-net-worth individuals, its success is highly dependent on personalized factors and optimal inve

In [ ]:
# for key, value in res.items():
#   print(f"{key}, {value}")

In [ ]:
# from pprint import pprint
# pprint(res)

In [ ]:
chain_map_reduce = load_summarize_chain(llm, chain_type="map_reduce")
res = chain_map_reduce.invoke(pages[0:16])
res
print(res["output_text"])

Vanguard's research from July 2024 underscores the importance of a personalized approach to tax-loss harvesting (TLH), revealing that its effectiveness can significantly vary based on investor characteristics, behaviors, and market conditions. The study, conducted by Thomas Paradise, CFA, Kevin Khang, Ph.D., and Joel M. Dickson, Ph.D., indicates that optimal TLH strategies, especially the reinvestment of tax savings, can enhance portfolio outcomes, potentially adding an annual after-tax return (TLH alpha) between 0.47% and 1.27%. This benefit largely depends on the proportion of taxable equity assets in an investor's portfolio and highlights the nuanced impact of TLH, advocating for tailored investment strategies.

The research elaborates on the three main drivers of TLH effectiveness: loss generation, conversion to tax savings, and increased market exposure through reinvestment of tax savings. It also discusses the risks and considerations of TLH, including wash sales and portfolio os

In [ ]:
from langchain.chains.summarize import load_summarize_chain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.docstore.document import Document
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

prompt = PromptTemplate(
    input_variables=["document"],
    template="""Write a summary for a minimum of 5 bullets for the following document:

    {document}
    Summary:
    """,
)


#define LLM chain
llm = ChatOpenAI(temperature=0.1, model_name="gpt-4-turbo-preview", api_key=open_ai_key)
summarization_chain = LLMChain(llm=llm, prompt=prompt)

stuff_chain = StuffDocumentsChain(
    llm_chain=summarization_chain,
    document_variable_name="document",
)

summary = stuff_chain.run(pages[0:16])
print(summary)

- The value of tax-loss harvesting (TLH) is highly dependent on individual investor characteristics, behaviors, and market conditions, each contributing roughly one-third to the variation in TLH outcomes. Optimal TLH behavior, particularly the reinvestment of tax savings, significantly enhances TLH benefits.
- TLH can yield an additional annual after-tax return (TLH alpha) ranging from 0.47% to 1.27%, depending on the investor's profile and the proportion of taxable equity assets in their portfolio. This emphasizes the importance of a personalized approach to TLH.
- The study underscores the importance of reinvesting tax savings from TLH into the portfolio as the most crucial behavior for maximizing TLH benefits. Factors such as volatility, harvest frequency, and market exposure also play significant roles in the effectiveness of TLH.
- Risks associated with TLH include the potential for wash sales, portfolio ossification, and the need for sufficient capital gains to convert harvested 

In [ ]:
!pip install -U langchain-anthropic pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 64.9 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.


In [ ]:
#claude_ai_key
claude_ai_key = userdata.get('claude_key')

In [ ]:
from langchain_anthropic import ChatAnthropic
from langchain.chains.summarize import load_summarize_chain
from langchain.docstore.document import Document
from langchain_core.messages import HumanMessage

# Initialize the model
model_claude = ChatAnthropic(model="claude-3-opus-20240229", temperature=0.1, api_key=claude_ai_key)

# Create a human message
#message = HumanMessage(content="Can you explain the concept of quantum entanglement?")

chain2 = load_summarize_chain(model_claude, chain_type="stuff")

res = chain2.invoke(pages[0:16])
res
print(res["output_text"])

Here is a concise summary of the key points from the Vanguard research on tax-loss harvesting (TLH):

- The value of TLH varies significantly based on investor characteristics, behavior, and market environment. Each of these three categories drives about one-third of the variation in value.

- Simulations showed TLH improved outcomes for every investor profile analyzed, relative to suboptimal behavior. The most important behavior driving TLH value is reinvesting the tax savings back into the portfolio. 

- TLH alpha, the potential additional annual after-tax return from TLH, ranged between 0.47% and 1.27% for the profiles analyzed. However, this must be scaled by the proportion of taxable equity assets in the total portfolio to understand the impact.

- Factors specific to the investor, factors the investor can influence, and factors outside the investor's control each account for roughly one-third of the TLH outcome. The biggest drivers are whether investors reinvest the tax savings a

In [ ]:
chain2 = load_summarize_chain(model_claude, chain_type="map_reduce")

res = chain2.invoke(pages[0:16])
res
print(res["output_text"])

Tax-loss harvesting (TLH) is a strategy used to improve after-tax returns by realizing capital losses to offset gains and income. The value of TLH depends on three key factors: generating losses through portfolio volatility, efficiently converting losses into tax savings, and reinvesting tax savings for long-term growth. Historical simulations and machine learning techniques reveal that the relative importance of these factors is roughly equal, with market exposure being the most crucial.

Investors can maximize TLH benefits by focusing on controllable factors such as recurring investments, portfolio granularity, harvest frequency, and reinvesting tax savings. However, TLH also comes with risks and limitations, including the wash sale rule, ossification, and the impact of market downturns.

The study examines TLH benefits across different investor profiles and market scenarios, finding that higher-net-worth investors benefit more from TLH, with optimal behaviors significantly impacting

In [ ]:
claude_prompt = PromptTemplate(
    input_variables=["document"],
    template="""Write a summary for a minimum of 5 bullets for the following document:

    {document}
    Summary:
    """,
)

llm_chain = LLMChain(llm=model_claude, prompt=claude_prompt)
claude_stuff_chain = StuffDocumentsChain(
    llm_chain=llm_chain,
    document_variable_name="document",
)

claude_summary = claude_stuff_chain.run(pages[0:16])
print(claude_summary)

InternalServerError: Error code: 529 - {'type': 'error', 'error': {'type': 'overloaded_error', 'message': 'Overloaded'}}